# Chapter 01

## Introduction

In [ ]:
# noinspection PyUnresolvedReferences
import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
# noinspection PyUnresolvedReferences
import scipy.stats as stats
# noinspection PyUnresolvedReferences
import seaborn as sns

# Import the `figsize` utility function
from IPython.core.pylabtools import figsize

In [ ]:
import fractions

### A farmer or a librarian?

In [ ]:
# 300 dots per inch for saved figures and inline plots
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['figure.dpi'] = 300

In [ ]:
prior = np.array([fractions.Fraction(1, 21), fractions.Fraction(20, 21)])
likelihood = np.array([fractions.Fraction(95, 100), fractions.Fraction(5, 10)])
products = prior * likelihood
posterior = products / sum(products)

In [ ]:
colors = ['#348abd', '#a60628']
plt.bar([0, 0.7], prior, alpha=0.7, width=0.25, color = colors[0],
        label='Prior distribution', lw='3', edgecolor=colors[0])
plt.bar([0 + 0.25, 0.7 + 0.25], posterior, alpha=0.7, width=0.25,
        color=colors[1], label='Posterior distribution',
        lw='3', edgecolor=colors[1])
plt.xticks([0.20, 0.95], ['Librarian', 'Farmer'])
plt.title("Prior and posterior probabilities of Steve's occupation.")
plt.ylabel('Probability')
plt.legend(loc='upper left')

### Distributions

A probability distribution is a function of a random variable, often
called _Z_, that maps _Z_ to a value such that the sum over all values of
_Z_ (either continuous or discrete) is one. That is, a probability
distribution is a function whose sum over all range values for each member
of the domain is exactly one.

#### Poisson distribution

A Poisson distribution is a discrete distribution (each member of the domain
is "distinct" or "seperated" from other members of the domain). The domain
of the Poisson distribution is the set of non-negative integers (infinite,
but countably infinite). The shape is controlled by a single (shape?)
parameter, _lambda_. Higher values of _lambda_ result in a higher probability
mass for larger elements of the domain.

The mean, first moment, or expectation of a Poisson distribution with
parameter, _lambda_, is $\lambda$.

In [ ]:
# Plot on a 12.5" x 4" canvas
plt.figure(figsize=(12.5, 4))
# Using two colors
colors = ['#348abd', '#a60628']

In [ ]:
# Plot two different Poisson distributions (probability mass functions)
a = np.arange(16)  # 16 x-values
poi = stats.poisson  # save a couple of keystrokes
# Note the trailing underscore because `lambda` is a keyword
lambda_ = [1.5, 4.25]

In [ ]:
# Calculate the probability "mass" of the Poisson distribution at each _a_
plt.bar(a, poi.pmf(a, lambda_[0]), color=colors[0],
        label=f'$\lambda = {lambda_[0]:.1f}$', alpha=0.60, edgecolor=colors[0], lw='3')
plt.bar(a, poi.pmf(a, lambda_[1]), color=colors[1],
        label=f'$\lambda = {lambda_[1]:.1f}$', alpha=0.60, edgecolor=colors[1], lw='3')
plt.xticks(a + 0.4, a)
plt.legend()
plt.ylabel('Probability of $k$')
plt.xlabel('$k$')
plt.title('Probability mass function of a Poisson random variable but'
          ' differing $\lambda$ values.')

#### Exponential distribution

An exponential distribution is a continuous distribution (members of the domain
are continuous). The domain of the exponential distribution is the set of
real numbers. The shape is controlled by a single (shape?) parameter, _lambda_.
Higher values of _lambda_ result in a higher probability mass for larger members
of the domain.

The mean, first moment, or expectation of an exponential distribution with
parameter, _lambda_, is $1 / \lambda$.

In [ ]:
a = np.linspace(0, 4, 100)  # Sample real numbers linearly
expo = stats.expon
lambda_ = [0.5, 1]

In [ ]:
for l, c in zip(lambda_, colors):
    plt.plot(a, expo.pdf(a, scale=1 / l), lw=3, color=c,
                         label=f'$\lambda$ = {l:.1f}')
    plt.fill_between(a, expo.pdf(a, scale=1 / l), color=c, alpha=0.33)
plt.legend()
plt.ylabel('Probability density function at $z$')
plt.xlabel('$z$')
plt.ylim(0, 1.2)
plt.title('Probability density function of an exponential random variable,'
          ' differing $\lambda$ values')

### Bayesian inference using computers

You are given a series of daily text message counts from a user of your
system. You are curious if the user's text messaging habits have changed
over time, either gradually or suddenly. How can you model this?

#### Plot the text message counts over time.

In [ ]:
# Plot on a 12.5" x 3.5" "canvas"
plt.figure(figsize=(12.5, 3.5))

In [ ]:
# Read in the data (single column text file, I believe)
count_data = np.loadtxt('data/txtdata.csv')
n_count_data = len(count_data)

In [ ]:
# Plot a bar chart: text message counts over time
plt.bar(np.arange(n_count_data), count_data, color='#348abd')
plt.xlabel('Time (days)')
plt.ylabel('Text messages received')
plt.title("Did the user's texting habits change over time?")
plt.xlim(0, n_count_data)

#### Modeling the counts over time

The author suggests modeling the counts using a Poisson distribution; that is,
$$
C_{i} \sim \mathrm{Poisson}(\lambda)
$$

However, $\lambda$, is a parameter whose value is **unknown**; consequently, we use Bayesian inference to infer a distribution of values for this parameter.

The author further posits that $\lambda$ actually changes (discontinuously) over time. That is, $\lambda$ is drawn from a _switchover_ at time, $\tau$.

$$
\lambda = \left\{
    \begin{array}\\
        \lambda_{1} \mbox{ if } \mathrm{t} < \tau \\
        \lambda_{2} \mbox{ if } \mathrm{t} \geq \tau
    \end{array}
\right.
$$

In order to infer the values $\lambda_{1}$ and $\lambda_{2}$, we must assign prior probabilities to these values. Because the $\lambda$ parameter for the Poisson distribution accepts any **positive** number, we need a distribution that generates positive numbers. The exponential distribution provides a continuous distribution for positive values so the author suggests using this distribution as follows:

$$
\begin{array}\\
    \lambda_{1} \sim \mathrm{Exp}(\alpha) \\
    \lambda_{2} \sim \mathrm{Exp}(\alpha)
\end{array}
$$

where \alpha is a _hyperparameter_; that is, a parameter that influences other **parameters**.

The author argues that setting $\alpha$ to the inverse of the sample average of the count data like

$$
\frac{1}{\mathrm{N}} \sum_{i = 0}^{\mathrm{N}} \mathrm{C}_{i} \approx \mathbb{E} [ \lambda\mid\alpha ] = \frac{1}{\alpha}
$$

The author does not detail his reasoning for this choice; however, the following explanation seems reasonable to me.

The counts are distributed as a Poisson distribution with parameter $\lambda$. The expectation of a Poisson distribution with parameter $\lambda$ is also $\lambda$. The sample mean of the counts is given by the summation expression in the previous section, and we want the sample mean to be the inverse of $\alpha$ but $\frac{1}{\alpha}$ is also the expectation of the parameter, $\lambda \mid \alpha$, for the exponential distribution.

The author states that this choice of hyperparameter is "not very opinionated in our prior"; that is, the prior minimizes the influence of the hyperparameter.

Finally, the author posits using a uniform distribution for $\tau$, the time at which the $\lambda$ parameter of the Poisson distribution changes. That is,

$$
\tau \sim \mathrm{DiscreteUniform}(0, 70)
$$

He argues that this choice is reasonable because of the noisiness of the data.

This distribution implies that

$$
\mathcal{P}(\tau = \mathcal{k}) = \frac{1}{70}
$$

## Introducing our first hammer: PyMC3

In [ ]:
alpha = 1.0 / count_data.mean()  ## `count_data` holds our text counts
with pm.Model() as model:
    # One must create all the stochastic variables in the context of a model
    lambda_1 = pm.Exponential('lambda_1', alpha)
    lambda_2 = pm.Exponential('lambda_2', alpha)
    tau = pm.DiscreteUniform('tau', lower=0, upper=n_count_data)

In [ ]:
with model:
    # We also define a deterministic `lambda_` inside a model
    idx = np.arange(n_count_data)
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2)

In [ ]:
# We add our observed data:
with model:
    obs = pm.Poisson('obs', lambda_, observed=count_data)

In [ ]:
# Finally, within a model, sample the data
with model:
    step = pm.Metropolis()
    trace = pm.sample(10000, tune=5000, step=step, return_inferencedata=False)

In [ ]:
lambda_1_samples = trace['lambda_1']
lambda_2_samples = trace['lambda_2']
tau_samples = trace['tau']

In [ ]:
# Plot the posterior samples
figsize(14.5, 10)

ax = plt.subplot(311)
ax.set_autoscaley_on(False)
plt.hist(lambda_1_samples, histtype='stepfilled', bins=30, alpha=0.85,
         label='posterior of $lambda_1$', color='#a60628', density=True)
plt.legend(loc='upper left')
plt.title(r"""Posterior distributions of the parameters $\lambda_1,\;\lambda_2,\;\tau$""")
plt.xlim([15, 30])
plt.xlabel('$\lambda_1$ value')

ax = plt.subplot(312)
ax.set_autoscaley_on(False)
plt.hist(lambda_2_samples, histtype='stepfilled', bins=30, alpha=0.85,
         label='posterior of $lambda_2$', color='#7a68a6', density=True)
plt.legend(loc='upper left')
plt.xlim([15, 30])
plt.xlabel('$\lambda_2$ value')
plt.ylabel('Density')

ax = plt.subplot(313)
w = 1 / tau_samples.shape[0] * np.ones_like(tau_samples)
plt.hist(tau_samples, bins=n_count_data, alpha=1, label='posterior of $\tau$',
         color='#467821', weights=w, rwidth=2.)
plt.xticks(np.arange(n_count_data))
plt.legend(loc='upper right')
plt.ylim([0, 0.75])
plt.xlim([35, len(count_data) - 20])
plt.xlabel(r'$\tau$ (in days)')
plt.ylabel('Probability')

In [ ]:
alpha = 1.0 / count_data.mean()  ## `count_data` holds our text counts
with pm.Model() as model:
    # One must create all the stochastic variables in the context of a model
    lambda_1 = pm.Exponential('lambda_1', alpha)
    lambda_2 = pm.Exponential('lambda_2', alpha)
    tau = pm.DiscreteUniform('tau', lower=0, upper=n_count_data)

    # We also define a deterministic `lambda_` inside a model
    idx = np.arange(n_count_data)
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2)

    # Add the observed data
    obs = pm.Poisson('obs', lambda_, observed=count_data)

    # And sample the data
    step = pm.Metropolis()
    idata = pm.sample(10000, tune=5000, step=step, return_inferencedata=True)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(16, 12), squeeze=False)
az.plot_posterior(idata, ax=ax)

## Interpretation

What have we gained:

- Uncertainty in our estimates
- Plausible values for parameters
- Posterior distributions of the two lambdas are clearly different
- Distribution of tau is narrow; that is, a small range for switch

## What good are samples from the posterior, anyway?

We'll use posterior samples to answer the following question: What is the expected number of texts at day, `t` $0 \leq t \leq 70$.

This question is equivalent to: What is the expected value of $\lambda$ at time $t$?

In the following code, let $i$ index samples from the posterior distributions. Given a day, $t$, we average over all possible $\lambda_i$ for that day, $t$, using $\lambda_i = \lambda_{1,i}$ if $\tau < \tau_i$ (that is, if the change has not yet occurred), else we use $\lambda_i = \lambda_{2,i}$.

In [ ]:
figsize(12.5, 5)

# tau_samples, lambda_1_samples, and lambda_2_samples each contain N samples
# from the corresponding posterior distribution.

N = tau_samples.shape[0]
expected_texts_per_day = np.zeros(n_count_data)

for day in range(n_count_data):
    # `ix` is a boolean index of all `tau_samples` corresponding to the
    # switch point occurring prior to the value of `day`.
    ix = day < tau_samples

    # Each posterior sample corresponds to a value for `tau`.
    #
    # For each day, that value of `tau` indicates whether we are "before"
    # (that is, in the `lambda_1` "regime") or "after" (that is, in the
    # "lambda_2" "regime") the switch point.
    #
    # By taking the posterior sample of `lambda_1` and `lambda_2` accordingly,
    # we can average over all samples to get an expected value for `lambda`
    # on that day.
    #
    # As explained, the "message count" random variable is Poisson-
    # distributed, and therefore, `lambda`, the Poisson parameter, is the
    # expected value of "message count."
    expected_texts_per_day[day] = (lambda_1_samples[ix].sum() +
                                   lambda_2_samples[~ix].sum()) / N

plt.plot(range(n_count_data), expected_texts_per_day, lw=4,
         color='#e24a33',
         label='Expected number of text messages received')
plt.xlim(0, n_count_data)
plt.xlabel('Day')
plt.ylabel('Number of text messages')
plt.title('Number of text messages received versus expected number received')
plt.ylim(0, 60)
plt.bar(np.arange(len(count_data)), count_data, color='#348abd',
        alpha=0.65, label='Observed text message per day')
plt.legend(loc='upper left')
print(expected_texts_per_day)

The author, in section 1.4.1, encourages the reader to try a different model with two different $\alpha$ values. Here's my attempt.

I'm going to split the original $\alpha$, set to the inverse of the mean of the counts, to be a "mixture" of two different means. The first mean is the average of the `count_data` over the first 50 days and the second mean is the average of the `count_data` over days 30 and beyond. This includes where I believe the change occurs, but instead of picking a specific day, overlaps the two averages.

In [ ]:
alpha_1 = 1.0 / count_data[:50].mean()  ## inverse of the average of the first 50 days of `count_data`
alpha_2 = 1.0 / count_data[30:].mean()  ## inverse of the average of the days 30 to the end of `count_data`
with pm.Model() as model:
    # One must create all the stochastic variables in the context of a model
    lambda_1 = pm.Exponential('lambda_1', alpha_1)
    lambda_2 = pm.Exponential('lambda_2', alpha_2)
    tau = pm.DiscreteUniform('tau', lower=0, upper=n_count_data)

    # We also define a deterministic `lambda_` inside a model
    idx = np.arange(n_count_data)
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2)

    # Add the observed data
    obs = pm.Poisson('obs', lambda_, observed=count_data)

    # And sample the data
    step = pm.Metropolis()
    idata = pm.sample(10000, tune=5000, step=step, return_inferencedata=True)

(Surprisingly) Nice that it sampled without warnings or errors.

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(16, 12), squeeze=False)
az.plot_posterior(idata, ax=ax)

I see some differences between the posterior plots; however, I do not know enough to judge if these differences are significant or could simply be the random samples that occurred during these two runs.

I wonder what the result is if I pick a "hard" but "incorrect" switchover date for $\alpha$. Hmmm. It's pretty easy to try. :)

In [ ]:
# I will pick the `alpha` change to occur on day 30. I do not believe this day, but I want to see what affect this has on the result.
alpha_1 = 1.0 / count_data[:30].mean()  ## inverse of the average of the first 30 days of `count_data`
alpha_2 = 1.0 / count_data[30:].mean()  ## inverse of the average of the days 30 to the end of `count_data`
with pm.Model() as model:
    # One must create all the stochastic variables in the context of a model
    lambda_1 = pm.Exponential('lambda_1', alpha_1)
    lambda_2 = pm.Exponential('lambda_2', alpha_2)
    tau = pm.DiscreteUniform('tau', lower=0, upper=n_count_data)

    # We also define a deterministic `lambda_` inside a model
    idx = np.arange(n_count_data)
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2)

    # Add the observed data
    obs = pm.Poisson('obs', lambda_, observed=count_data)

    # And sample the data
    step = pm.Metropolis()
    idata = pm.sample(10000, tune=5000, step=step, return_inferencedata=True)

That behavior is surprising. I expected my hard cut over to make a difference, but it appears to have little effect.

As a final experiment, instead of specifying the step, I will use the approach recommended by the PyMC3 documentation and simply sample the model.

In [ ]:
alpha = 1.0 / count_data.mean()  ## `count_data` holds our text counts
with pm.Model() as model:
    # One must create all the stochastic variables in the context of a model
    lambda_1 = pm.Exponential('lambda_1', alpha)
    lambda_2 = pm.Exponential('lambda_2', alpha)
    tau = pm.DiscreteUniform('tau', lower=0, upper=n_count_data)

    # We also define a deterministic `lambda_` inside a model
    idx = np.arange(n_count_data)
    lambda_ = pm.math.switch(tau > idx, lambda_1, lambda_2)

    # Add the observed data
    obs = pm.Poisson('obs', lambda_, observed=count_data)

    # And sample the data
    step = pm.Metropolis()
    idata = pm.sample(return_inferencedata=True)

In [ ]:
fig, ax = plt.subplots(3, 1, figsize=(16, 12), squeeze=False)
az.plot_posterior(idata, ax=ax)

Interesting.

None of the posteriors seem significantly different in terms of the mean or the 94% HDI; however, I originally thought that the $\tau$ posterior was "fatter" when I let the software sample. However, as I considered this hypotheses, I'm less convinced. The software sampling for $\tau$  seems to cut off at 41.5 on the low side; when I specified the sampling, it appeared to be zero below 41, but the total left-hand tail is perhaps longer (down to 38).

I do not know enough about the details to decide if the story I believe is what the picture is actually communicating. Good I have future questions, but bad that I do not have "an answer."

## Appendix

### Determining if the two $\lambda$s are different

Determining if the posterior distributions of two different variables (parameters) is fairly straightforward.
One merely need to computer the fraction of the time that $\lambda_1$ is less than $\lambda_2$.

In [ ]:
less_than = lambda_1_samples < lambda_2_samples
# `sum` counts the number of True values (because Python treats `True` as 1)
less_than_fraction = less_than.sum() / len(less_than)
less_than_fraction

Although absolutely true (1.0) is unexpected, the value reported by the book is 29994 / 30000; that is, very close
to 1. I can easily see that another simulation might generate exactly one.

In fact, the book does not actually calculate the fraction, but uses `mean()` which uses the same "trick" described
in the book, _Thinking Bayes_, 2nd edition.

In [ ]:
less_than.mean()

We can also ask more complicated questions such as, "What is the probability that $\lambda_1$ - $\lambda_2$ is
greater than 1? 2? 5? 10?

In [ ]:
for greater_than_difference in [0.1, 0.2, 0.3, 0.5, 0.8, 1.3, 2.1, 3.4, 5.5, 8.9, 14.4]:
    print(f'Probability that abs(lambda_1 - lambda_2) > {greater_than_difference}:'
          f' {(abs(lambda_1_samples - lambda_2_samples) > greater_than_difference).mean()}')

What if we extend the model to *two* switchpoints?

In [ ]:
alpha = 1.0 / count_data.mean()  ## `count_data` holds our text counts
with pm.Model() as three_switch_points:
    # One must create all the stochastic variables in the context of a model
    lambda_1 = pm.Exponential('lambda_1', alpha)
    lambda_2 = pm.Exponential('lambda_2', alpha)
    lambda_3 = pm.Exponential('lambda_3', alpha)
    tau_1 = pm.DiscreteUniform('tau_1', lower=0, upper=n_count_data - 1)
    tau_2 = pm.DiscreteUniform('tau_2', lower=tau_1, upper=n_count_data)


    # We also define a deterministic `lambda_` inside a model
    idx = np.arange(n_count_data)
    # The following, commented out expression produces a `Type` error with the message: "Variables do not support
    # boolean operations." I believe the explanation of this error is found on the
    # [PyMC3 Discourse post](https://github.com/Theano/Theano/issues/1581). Basically, Theano variables are
    # symbolic even though they appear to be Python variables.
    # lambda_ = pm.math.switch(idx < tau_1, lambda_1,
    #                          pm.math.switch(tau_1 <= idx < tau_2, lambda_2, lambda_3))
    lambda_ = pm.math.switch(idx <= tau_1, lambda_1,
                             pm.math.switch(idx > tau_2, lambda_3, lambda_2))

    # Add the observed data
    obs = pm.Poisson('obs', lambda_, observed=count_data)

    # And sample the data
    # The following code results in issues: a `RuntimeWarning` about invalid values and a warning that the
    # number of effective samples is smaller than 10% for some parameters. I tried the "default" `sample`
    # method instead.
    step = pm.Metropolis()
    idata = pm.sample(10000, tune=5000, step=step, return_inferencedata=True)
    # However, the "default" `sample` method produces similar warnings.
    # idata = pm.sample(return_inferencedata=True)

In [ ]:
fig, ax = plt.subplots(5, 1, figsize=(12, 20), squeeze=False)
az.plot_posterior(idata, ax=ax)